# Starting Out With Bioimage Processing in Python
*Author: Vladislav Kim*
* [Introduction](#intro)
* [Bioimage formats and loading](#load)
* [Viewing Images in Jupyter](#view)
* [Handling multichannel images with $z$-stack](#compleximages)
* [Applications to high-content screening](#hcs)
* [Writing images](#writeimg)


<a id="intro"></a> 
## Introduction
In this notebook we show how to load and visualize microscopy images in Python. There is a vast number of microscopy image formats in use and with the help of `python-bioformats` we can effortlessly load these in Python. We will demonstrate a number of use cases for `base.utils` module which has a number of useful functions for visualizing, reading and writing complex images (multichannel, 3D, etc).


The first step before running this notebook would be to set up a conda environment with all the dependencies (see [README](https://github.com/vladchimescu/bioimg/blob/master/README.md)). Once the environment is set up, activate it and start jupyter server in the activated environment.


In [ ]:
# load third-party Python modules
import javabridge
import bioformats as bf
import skimage
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('..')

javabridge.start_vm(class_path=bf.JARS)

<a id="load"></a> 
## Bioimage formats and data loading
Bio-Formats library, developed by Open Microscopy Environment (OME), supports 150 bioimage formats. Some of the supported formats include common image file extensions such as TIFF, JPG and PNG, but can handle some proprietary microscopy formats such as Zeiss CZI, Leica LCF, Canon DNG, etc. For the full list of supported image formats refer to [this page](https://docs.openmicroscopy.org/bio-formats/6.3.1/supported-formats.html).


 We provide a number of example images that can be downloaded (see [download.md](https://github.com/vladchimescu/bioimg/blob/master/Jupyter/data/download.md)). We will start out with images of leukemia and stroma nuclei (`data/CLL-coculture` after you downloaded the files from Dropbox). The imaged well has 2 color channels: Hoechst for nucleus staining and a dye that marks lyosomal compartments. 

The first step is to load microscopy images. The images that we will be working with are in TIFF format. In the local module `base.utils` we provide a function `read_image` which is a wrapper that reads 2D images (in any of the OME-supported formats) and outputs a `numpy.array` object:

In [ ]:
from base.utils import read_image
img_ho = read_image(fname='data/CLL-coculture/r01c02f01-Hoechst.tiff')

Two-dimensional images can be represented as 2D numerical arrays (`np.array`) or matrices:

In [ ]:
print(type(img_ho))
print(img_ho.shape)

<a id="view"></a> 
## Viewing Images in Jupyter
We can plot the image arrays using `mapltolib` as grey-scale images:

In [ ]:
plt.figure(figsize=(7,7))
plt.imshow(img_ho)
plt.axis('off')

If a microscopy image has several color channels, these can be plotted individually as grey-scale images side by side. Load another chanel of the same well:

In [ ]:
img_ly = read_image(fname='data/CLL-coculture/r01c02f01-Ly.tiff')

To view the color channels side by side use `plot_channels` from `base.plot`:

In [ ]:
from base.plot import plot_channels
plot_channels([img_ho, img_ly], titles=['Nuclei', 'Lysosomes'], nrow=1, ncol=2)

You can combine channels in a single image, use `combine_channels` function and specify the colors for each channel:

In [ ]:
from base.plot import combine_channels
# here we use gamma correction for 'img_ho'
img_overlay = combine_channels([img_ho**0.5, img_ly],
                               colors=['blue', 'white'],
                               blend = [1.5, 0.7])

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_overlay)
plt.axis('off')

<a id="compleximages"></a> 
## Handling compound multichannel images (high-content screening)
In addition to color information, microscopy images may have optical sections along the $z$-axis. Handling 3D multichannel data is trivial in Python, as these can be represented as (3D+color)-`np.array`. We can load one such image using `load_imgstack` function

In [ ]:
from base.utils import load_imgstack
imgstack = load_imgstack(fname="data/BiTE/Tag2-r04c02f1.tiff")

Usually the first dimension is reserved for optical sections ($z$-stack), the next two dimensions describe image coordinates ($xy$-plane) and the last dimension is to indicate color channel. The `imgstack` array uses precisely this order.

First we can apply maximum intensity projection (MIP) to aggregate images along the $z$-direction and make them two-dimensional (+ color)

In [ ]:
mip = np.amax(imgstack, axis=0)
print(mip.shape)

As mentioned before channels are in the last array axis (dimension), we can split the color channels and plot them side by side:

In [ ]:
# split individual color channels and place them in a list
mip_split = [mip[:,:,i] for i in range(mip.shape[2])]

In [ ]:
plot_channels(mip_split,
              nrow=1, ncol=4,
              titles=['CD20+', 'Calcein',
                      'Nuclei', 'CD8+'])

In [ ]:
mip_color = combine_channels(mip_split, 
                             colors=['red', 'green',
                                     'blue','orange'],
                             # these are optional (see documentation)
                             blend = [0.8, 0.8, 2, 0.8],
                             gamma = [0.3, 0.3, 0.4, 0.3])

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(mip_color)
plt.axis('off')

*You can skip 'Applications to High-Content Screening' upon first reading*
<a id="hcs"></a> 
## Applications to High-Content Screening
Some microscopes output a series of images instead of a single image stack. We can use the function `base.utils.load_image_series` to load all the color channels and $z$-stack in a single `numpy.array`.


Here we will load a series of images from a high-content screen. Wells of a 384-well plate are numbered (r01 = row 1, c16 = column 16) and we would like to load a single well that has 
+ 3 color channels
+ 7 optical sections ($z$-stack)
+ 3 fields of view (sampled positions in the $xy$-plane at which the well was imaged)

Suppose we want to load well 'r01c02' and only the first field of view ('f01'):

In [ ]:
# list files
import os
files = os.listdir('data/AML_screen')
print(files[:5])

In [ ]:
# well r01c02, position 1 (f01)
import re
wellfiles = [re.search('r01c02f01.+', f).group() for f in files
                if re.search('r01c02f01', f)]
# sort them lexicographically
wellfiles.sort()

print(wellfiles[:5])

In [ ]:
from base.utils import load_image_series
imgseries = load_image_series(path='data/AML_screen', imgfiles=wellfiles)

Before reading in images, `load_image_series` sorts the file names lexicographically so that the array is filled in the right order. Here the images are first sorted by `p[0-9` ($z$-position) and then by color channel. We can reshape the array to have 10 $z$-sections, 3 color channels and the width (2160) and height (2160).

In [ ]:
imgseries = imgseries.reshape((10, 3, 2160,2160))

We can get rid of the $z$-axis by taking maximum value across all the $z$-sections -- the so-called maximum intensity projection (MIP)

In [ ]:
mipseries = np.amax(imgseries, axis=0)

Now we've got an array with 3 images corresponding to individual color channels:

In [ ]:
# plot 3 channels side by side
plot_channels([mipseries[i] for i in range(3)], nrow=1, ncol=3)

These can be combined in a single image:

In [ ]:
rgbseries = combine_channels([mipseries[i] for i in range(3)],
                            colors=['blue', 'red', 'green'],
                            blend=[1.5,1.5,2],
                            gamma=[0.6, 0.6,0.6])

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(rgbseries)
plt.axis('off')

<a id="writeimg"></a> 
## Writing images
We can save processed images in OME-supported formats: 

In [ ]:
# apply gamma correction
img_out = img_ho**0.5

In [ ]:
from base.utils import write_image
write_image(img_out, path='data/Hoechst-image.png')

To save an image stack:

In [ ]:
imgstack_out = np.swapaxes(mipseries, 0, 2)

In [ ]:
from base.utils import write_imgstack
write_imgstack(img=imgstack_out, path='data/MIP-AML.tiff',
           size_z=1, size_c=3)

In [ ]:
javabridge.kill_vm()

In the [next notebook](https://github.com/vladchimescu/bioimg/blob/master/Jupyter/2-image-transformation.ipynb) we will show how images can be preprocessed using various transformations avaialbe in `scikit-image` library before downstream analysis, such as segmentation, is run.